In [51]:
from IPython.display import display, HTML

import json
import requests
from jupyter_widget_stixview.widget import StixviewGraph

from stix2 import (
    Bundle, Indicator, Malware, Relationship,
    AttackPattern, Campaign, ThreatActor)


narrator_api = "https://polzunov.com/narrator/report/"


def as_bundle(objects):
    bundle_obj = Bundle(*objects)
    return json.loads(bundle_obj.serialize())


def get_report(data):
    r = requests.post(narrator_api, json=data)
    r.raise_for_status()
    response = r.json()
    return response['text']


def render_html(html):
    display(HTML(html))

In [54]:
indicator = Indicator(
    name="File hash for malware variant X",
    labels=["malicious-activity"],
    pattern="[file:hashes.md5 = 'd41d8cd98f00b204e9800998ecf8427e']")

malware = Malware(
    name="Malware Variant X",
    labels=['remote-access-trojan'])

rel1 = Relationship(
    source_ref=indicator.id,
    relationship_type='indicates',
    target_ref=malware.id)

campaign = Campaign(name="Campaign Y")

rel2 = Relationship(
    source_ref=campaign.id,
    relationship_type='uses',
    target_ref=malware.id)

actor = ThreatActor(
    name='Actor Z',
    labels=['criminal'])

rel3 = Relationship(
    source_ref=campaign.id,
    relationship_type='attributed-to',
    target_ref=actor.id)

objects = [indicator, malware, rel1, campaign, rel2, actor, rel3]

StixviewGraph(
    bundle=as_bundle(objects),
    properties={"graphHeight": 300})

StixviewGraph(bundle={'objects': [{'name': 'File hash for malware variant X', 'labels': ['malicious-activity']…

In [55]:
render_html(get_report({
    "bundle": as_bundle(objects),
    "type": "timeline",
    "seed": None,
}))

In [56]:
render_html(get_report({
    "bundle": as_bundle(objects),
    "type": "entity-overview",
    "seed": str(malware.id),
}))